In [6]:
import pandas as pd
from datetime import datetime
import requests

In [7]:
# Displaying some Crash List Data
df_crash_list = pd.read_csv("../Resources/CrashList.csv",index_col=0)
df_crash_list = df_crash_list[df_crash_list["CrashDate"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%Y")) != "2017"]

<h3>2. Latitude and Longitude of each unique (county and state)</h3>

In [ ]:
# Get rid of parentheses and the content in them with regular expression
df_locations_fatalities = df_crash_list[["CountyName", "StateName", "Fatals"]].copy()
df_locations_fatalities["CountyName"] = df_locations_fatalities["CountyName"].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
df_locations_fatalities.head()

In [ ]:
# Find longitude and latitude of each state and county
df_fatalities_by_loc = df_locations_fatalities.groupby(["CountyName", "StateName"]).count()

# Convert it back to DF
df_fatalities_by_loc = df_fatalities_by_loc.reset_index()


# For each unique location, find longitude and latitude using geoapify
for idx, location in df_fatalities_by_loc.iterrows():

    base_url = "https://api.geoapify.com/v1/geocode/search?"
    
    county = location["CountyName"]
    state = location["StateName"]
    
    params = {
        "text": f"{county}, {state}",
        "apiKey": ""
    }

    try:
        res = requests.get(base_url, params=params)
        data = res.json()

        properties = data["features"][0]["properties"]

        lat = properties["lat"]
        lon = properties["lon"]
        
        df_fatalities_by_loc.at[idx, "lat"] = lat
        df_fatalities_by_loc.at[idx, "long"] = lon
    except Exception as e:
        print("Oh no, we ran into a problem:", e)

df_fatalities_by_loc



In [ ]:
#Save as CSV
df_fatalities_by_loc.to_csv("../Resources/County_State_Fatalities_18-19.csv")